##### Module import

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor # 다중공선성

In [2]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

##### Data import

In [3]:
print(os.getcwd())

c:\Users\Owner\Desktop\윤태준\고용패널\고용패널 파이썬 분석\2023_04_06_변수추가\변수추가


In [4]:
df = pd.read_csv('2023_04_10_data_step.csv', encoding='cp949')

In [5]:
df_1 = df.copy()
df_1.columns

Index(['pid', 'majorcat', 'province', 'area', 'sex', 'age', 'a003', 'a010',
       'a038', 'a039',
       ...
       'big_companyG', 'work_satiG', 'company_satiG', 'supply', 'insurance',
       '주당근로일', '주당근로시간', '직장에서의복리후생제공여부', '보험가입여부', '직장만족도'],
      dtype='object', length=106)

##### 다변량 모델

#### 복지 모델

In [6]:
model_1 = sm.Logit.from_formula('''LongWork2 ~ C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_1.summary()

Optimization terminated successfully.
         Current function value: 0.544362
         Iterations 5
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5805
Method:                           MLE   Df Model:                            2
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                 0.06165
Time:                        20:22:17   Log-Likelihood:                -3161.7
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                 6.229e-91
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -1.6740      0.055    -30.503      0.000      -1.782      -1.566
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     1.0717      0.069     15.474      0.000       0.936       1.207
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.6123      0.067      9.104      0.000       0.481       0.744
==================================================================================================================
"""

In [7]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_1 = pd.DataFrame(
    {
        "OR": model_1.params,
        "Lower CI": model_1.conf_int()[0],
        "Upper CI": model_1.conf_int()[1],
    }
)
odds_ratios_1 = np.exp(odds_ratios_1)
odds_ratios_1

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.187490,0.168370,0.208782
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.920222,2.549571,3.344757
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.844752,1.616896,2.104717


In [8]:
model_2 = sm.Logit.from_formula('''LongWork2 ~ + C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    +  C(보험가입여부,Treatment('가입하지않음')) + 주당근로시간''', df_1).fit()
print('==================================Model Summary=========================================')
model_2.summary()

Optimization terminated successfully.
         Current function value: 0.539437
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5804
Method:                           MLE   Df Model:                            3
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                 0.07014
Time:                        20:22:24   Log-Likelihood:                -3133.0
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                4.083e-102
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -2.4188      0.119    -20.292      0.000      -2.652      -2.185
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     1.0437      0.069     15.061      0.000       0.908       1.180
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.4319      0.071      6.106      0.000       0.293       0.571
주당근로시간                                             0.0232      0.003      7.417      0.000       0.017       0.029
==================================================================================================================
"""

In [9]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_2 = pd.DataFrame(
    {
        "OR": model_2.params,
        "Lower CI": model_2.conf_int()[0],
        "Upper CI": model_2.conf_int()[1],
    }
)
odds_ratios_2 = np.exp(odds_ratios_2)
odds_ratios_2

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.089032,0.070483,0.112463
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.839760,2.479109,3.252877
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.540141,1.340775,1.769152
주당근로시간,1.023425,1.017182,1.029706


In [10]:
model_3 = sm.Logit.from_formula('''LongWork2 ~ 
    + C(직장에서의복리후생제공여부,Treatment('제공하지않는다'))
    + C(보험가입여부,Treatment('가입하지않음'))
    + 주당근로시간 + C(직장만족도,Treatment('그렇지않다'))''', df_1).fit()
print('==================================Model Summary=========================================')
model_3.summary()

Optimization terminated successfully.
         Current function value: 0.520202
         Iterations 6
==================================Model Summary=========================================


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              LongWork2   No. Observations:                 5808
Model:                          Logit   Df Residuals:                     5802
Method:                           MLE   Df Model:                            5
Date:                Tue, 11 Apr 2023   Pseudo R-squ.:                  0.1033
Time:                        20:24:50   Log-Likelihood:                -3021.3
converged:                       True   LL-Null:                       -3369.4
Covariance Type:            nonrobust   LLR p-value:                3.505e-148
==================================================================================================================
                                                     coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                         -3.4184      0.152    -22.460      0.000      -3.717      -3.120
C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]     0.9074      0.071     12.747      0.000       0.768       1.047
C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]             0.4681      0.072      6.496      0.000       0.327       0.609
C(직장만족도, Treatment('그렇지않다'))[T.그렇다]                1.2579      0.095     13.304      0.000       1.073       1.443
C(직장만족도, Treatment('그렇지않다'))[T.보통이다]               0.5859      0.099      5.938      0.000       0.393       0.779
주당근로시간                                             0.0290      0.003      9.012      0.000       0.023       0.035
==================================================================================================================
"""

In [11]:
print('==================================Odds Ratio=========================================')
# 오즈비 계산(Confidence interval 포함)
odds_ratios_3 = pd.DataFrame(
    {
        "OR": model_3.params,
        "Lower CI": model_3.conf_int()[0],
        "Upper CI": model_3.conf_int()[1],
    }
)
odds_ratios_3 = np.exp(odds_ratios_3)
odds_ratios_3

==================================Odds Ratio=========================================


,OR,Lower CI,Upper CI
Intercept,0.032766,0.024314,0.044154
"C(직장에서의복리후생제공여부, Treatment('제공하지않는다'))[T.제공한다]",2.477768,2.155117,2.848726
"C(보험가입여부, Treatment('가입하지않음'))[T.가입했다]",1.596935,1.386601,1.839175
"C(직장만족도, Treatment('그렇지않다'))[T.그렇다]",3.517925,2.922834,4.234177
"C(직장만족도, Treatment('그렇지않다'))[T.보통이다]",1.796595,1.480702,2.179882
주당근로시간,1.029374,1.022913,1.035876
